In [2]:
import json
import pandas as pd
from pandas.io.json import json_normalize #package for flattening json in pandas df
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
with open('C:\\Users\\chq-shal\\Downloads\\projects_python\\project_workboard\\small3.json',encoding='utf-8') as d:
    f=json.load(d)
data=json_normalize(f)

In [20]:
list(data)

['beginTime', 'endTime', 'payload', 'scenarioBit', 'visibility']

In [3]:
begin=pd.to_datetime(data.beginTime, utc=True)
end=pd.to_datetime(data.endTime, utc=True)
compliant=end-begin<np.timedelta64(2,'h') # true is compliant, false is not compliant

In [4]:
payload=data['payload'].apply(lambda row:pd.Series(json.loads(row))) # convert payload dictionaries to columns
visibility=data['visibility'].apply(lambda row:pd.Series(json.loads(row))) # convert visibility dictionaries to columns

In [21]:
df=pd.concat([begin,payload,data.scenarioBit,visibility],axis=1)
df=df.dropna(axis=1) # drop columns that have na values in them
df=df.drop(['events','firstAware','etmsBranch','workBranch'],axis=1) # drop dict events, firstAware; drop dup etms, work
df=df.loc[:,~df.columns.duplicated()] # drop dup columns

In [22]:
df['dateDT']=pd.to_datetime(df.dateCreate, utc=True)
df['diffS']=(df.dateDT-df.beginTime).dt.total_seconds()
df['beginY']=df.beginTime.dt.dayofyear
df['beginW']=df.beginTime.dt.dayofweek
df['beginH']=df.beginTime.dt.hour
df['dateY']=df.dateDT.dt.dayofyear
df['dateW']=df.dateDT.dt.dayofweek
df['dateH']=df.dateDT.dt.hour

In [23]:
too_sim=0.0003*len(df) # very similar values in a column
too_dif=0.5*len(df) # very different values in a column
for col in df.columns: # remove columns that have values that are too similar or too different
    if df[col].nunique()<too_sim or df[col].nunique()>too_dif:
        df.drop(col,inplace=True,axis=1)

In [30]:
df['scenarioBit'] = df['scenarioBit'].astype('category')

In [31]:
X_train,X_test,y_train,y_test=train_test_split(pd.get_dummies(df),compliant,random_state=0)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [32]:
from sklearn.ensemble import GradientBoostingClassifier
# clf=GradientBoostingClassifier(max_depth=2,random_state=0)
# grid_values = {'learning_rate': [0.001, 0.01, 0.05, 0.1, 1, 10]}
# grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
# grid_clf_auc.fit(X_train_scaled, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test_scaled)
# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc.best_score_)
clf=GradientBoostingClassifier(learning_rate=1,max_depth=2,random_state=0) # build bradient boost model with learning rate=1
gb_clf=clf.fit(X_train_scaled, y_train)
gb_y_score=gb_clf.decision_function(X_test_scaled)
gb_fpr, gb_tpr, _ = roc_curve(y_test, gb_y_score)
gb_roc_auc = auc(gb_fpr, gb_tpr)
gb_roc_auc

0.9204032723201823

In [33]:
feature=gb_clf.feature_importances_
pd.Series(data=feature,index=list(X_train)).nlargest(5)

shipmentType_OHA     0.335908
diffS                0.095452
portOrigin_HKG       0.092382
portOrigin_SFO       0.046075
scenarioBit_16384    0.029829
dtype: float64

In [34]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier()
# grid_values = {'n_estimators': [200, 700],'max_features': ['auto', 'sqrt', 'log2']}
# grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
# grid_clf_auc.fit(X_train_scaled, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.predict_proba(X_test_scaled)[:,1]
# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc.best_score_)
clf=RandomForestClassifier(max_features='log2', n_estimators=700)
rf_clf=clf.fit(X_train_scaled, y_train)
rf_y_score=rf_clf.predict_proba(X_test_scaled)[:,1]
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_y_score)
rf_roc_auc = auc(rf_fpr, rf_tpr)
rf_roc_auc

0.9571210702054878

In [35]:
feature=rf_clf.feature_importances_
pd.Series(data=feature,index=list(X_train)).nlargest(5)

diffS               0.112372
beginH              0.092069
dateH               0.084104
beginY              0.047342
shipmentType_OHA    0.043301
dtype: float64

In [36]:
from sklearn.neighbors import KNeighborsClassifier
# clf = KNeighborsClassifier(n_jobs=-1)
# grid_values = {'n_neighbors': list(range(1,11))}
# grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
# grid_clf_auc.fit(X_train_scaled, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.predict_proba(X_test_scaled)[:,1]
# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc.best_score_)
clf=KNeighborsClassifier(n_neighbors=7,n_jobs=-1) # build knn model with neighbors=7
knn_clf=clf.fit(X_train_scaled, y_train)
knn_y_score=knn_clf.predict_proba(X_test_scaled)[:,1]
knn_fpr, knn_tpr, _ = roc_curve(y_test, knn_y_score)
knn_roc_auc = auc(knn_fpr, knn_tpr)
knn_roc_auc

0.9066886870355079

In [37]:
from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(n_jobs=-1)
# grid_values = {'C': [1,10,100,1000]}
# grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
# grid_clf_auc.fit(X_train_scaled, y_train)
# y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test_scaled)
# print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
# print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
# print('Grid best score (AUC): ', grid_clf_auc.best_score_)
clf=LogisticRegression(C=1,n_jobs=-1) # build logistic regression model with C=1
lr_clf=clf.fit(X_train_scaled, y_train)
lr_y_score=lr_clf.predict_proba(X_test_scaled)[:,1]
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_y_score)
lr_roc_auc = auc(lr_fpr, lr_tpr)
lr_roc_auc

C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chq-shal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.8845102656352887

In [38]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
dt_clf=clf.fit(X_train_scaled, y_train)
dt_y_score=dt_clf.predict_proba(X_test_scaled)[:,1]
dt_fpr, dt_tpr, _ = roc_curve(y_test, dt_y_score)
dt_roc_auc = auc(dt_fpr, dt_tpr)
dt_roc_auc

0.8677800375241091